In [1]:
#imports parameters file, which contains login information
import parameters as p
#Web Scraping Tools
from time import sleep
from selenium import webdriver
from bs4 import BeautifulSoup as bs
#DataFrame & Excel Exporting Tools
import pandas as pd
import tkinter as tk
from tkinter import filedialog as fd

In [2]:
class Scrape:
    def __init__(self, c):
        #utilize webdriver to log into LinkedIn
        driver = webdriver.Chrome('/Users/shailesh/Documents/chromedriver')
        self.driver = driver
        
        self.driver.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

        #p.ln_user = LN username
        username = self.driver.find_element_by_name('session_key')
        username.send_keys(p.ln_user)

        #p.ln_password = LN password
        username = self.driver.find_element_by_name('session_password')
        username.send_keys(p.ln_ps)

        #clicks submit button for login
        login = self.driver.find_element_by_class_name('login__form_action_container')
        login.click()
        
        if(c == 0):
            #initializes lists for dataframe 
            self.name = []
            self.url = []
            self.city = []
            self.state = []
            self.geoID = []
            self.fYear = []
            self.numEmployee = []
            self.numLNE = []
            self.overview = []
            self.industry = []
            self.specs = []
            self.numRounds = []
            self.totAmt = []
            self.fundID = []

        #call Retrieve function
        self.retrieve()
        
    def retrieve(self):
        #name = ['hive9', 'Lucd','Loadbalancer-org', 'ParkMyCloud', 'Rigado', 'FileCloud']
        #name is input for company name
        #can be replaced with an array and iterated through
        
        #single input
        #url = input("Company URL(s) | If done, enter -1): ")
        
        #multiple input
        links = list(map(str, input("Enter Company URL(s): ").split()))
        links.append(-1)
        #c = 0
        
        for url in links:
            if(url == -1):
                self.driver.quit()
                choice = input('Select 1 to export current data to CSV | 2 to add onto an existing CSV | Anything else to exit\n')
                df = pd.DataFrame(list(zip(self.name, self.url, self.city, self.state, self.geoID, self.fYear, self.numEmployee, self.numLNE, self.overview, self.industry, self.specs, self.numRounds, self.totAmt, self.fundID)),
                                 columns = ['Name', 'URL','City', 'State', 'Geo ID' ,'Founding Year', 'Number of Employees', 'Number of LinkedIn Employees', 'Overview', 'Industry', 'Specifications', 'Number of Funding Rounds', 'Total Funding Amount', 'Fund ID'])
                print(df.head())
                if(choice == '1'):
                    #df.to_csv(r'/Users/shailesh/Programs/scrper/companies.csv', index=False, header=True)
                    root = tk.Tk()
                    c1 = tk.Canvas(root, width = 300, height = 300, bg = 'lightsteelblue2', relief = 'raised')
                    c1.pack()
                    
                    def exportCSV():
                        export_file_path = fd.asksaveasfilename(defaultextension='.csv')
                        df.to_csv (export_file_path, index = False, header=True)
                    
                    saveAsButton_CSV = tk.Button(text='Export CSV', command=exportCSV, bg='green', fg='white', font=('helvetica', 12, 'bold'))
                    c1.create_window(150, 150, window=saveAsButton_CSV)

                    root.mainloop()
                    
                    exit()
                elif(choice == '2'):
                    df.to_csv(r'/Users/shailesh/Programs/scrper/companies.csv', mode='a',index=False, header=False)
                else:
                    exit()
            else:
                self.driver.get(url)
                r = self.driver.page_source
                #utilizes beautifulsoup to sift through source code
                soup = bs(r, 'html5lib')

                #Company Name
                name = soup.find('h1', attrs={'class': 'org-top-card-summary__title t-24 t-black truncate'}) ['title']
                #print('Name: ' + name)
                self.name.append(name)
                
                #URL
                self.url.append(url)
                
                #headquarters
                hq = soup.find_all('div', attrs={'class': 'org-top-card-summary-info-list__info-item'})[1].getText()
                hq = hq.strip()
                hp = list(hq.split(", "))
                #print('City: ' + hp[0])
                self.city.append(hp[0])
                #print('State: ' + hp[1])
                self.state.append(hp[1])
                
                #automatic state disqualification parameter
                #1 if Company HQ State falls within CA, NY, FL, MO, HI, or MA
                if(hp[1] == 'CA' or hp[1] == 'NY' or hp[1] == 'MO' or hp[1] == 'HI' or hp[1] == 'FL' or hp[1] == 'MA' or hp[1] == 'California' or hp[1] == 'New York' or hp[1] == 'Missouri' or hp[1] == 'Hawaii' or hp[1] == 'Florida' or hp[1] == 'Massachusetts'):
                    self.geoID.append(1)
                else:
                    self.geoID.append(0)
        
                #year founded
                yearFounded = soup.find_all('dd', attrs={'class': 'org-page-details__definition-text t-14 t-black--light t-normal'})[-2].getText()
                yearFounded = yearFounded.strip()
                #print('Founding Year: ' + yearFounded)
                self.fYear.append(yearFounded)
                
                #num of employees
                numEmployees = soup.find('dd', attrs={'class': 'org-about-company-module__company-size-definition-text t-14 t-black--light mb1 fl'}).getText()
                numEmployees = numEmployees.strip()
                #print('Number of Employees: ' + numEmployees)
                self.numEmployee.append(numEmployees)

                #num of LN Employees
                lnEmployees = soup.find('dd', attrs={'class': 'org-page-details__employees-on-linkedin-count t-14 t-black--light mb5'}).getText()
                lnEmployees = lnEmployees.strip()
                self.numLNE.append(lnEmployees)
                
                #overview
                overview = soup.find('p', attrs={'class': 'break-words white-space-pre-wrap mb5 t-14 t-black--light t-normal'}).getText()
                overview = overview.strip()
                #print('Overview: \n' + overview)
                self.overview.append(overview)
                
                #industry
                industry = soup.find_all('div', attrs={'class': 'org-top-card-summary-info-list__info-item'})[0].getText()
                industry = industry.strip()
                #print('\nIndustry: ' + industry)
                self.industry.append(industry)

                #specialties
                specs = soup.find_all('dd', attrs={'class': 'org-page-details__definition-text t-14 t-black--light t-normal'})[-1].getText()
                specs = specs.strip()
                #print('Specifications: ' + specs)
                self.specs.append(specs)

                #capital raised $ & round type
                try:
                    cb_link = soup.find('a', attrs={'data-control-name': 'funding_all_rounds_link'}) ['href']
                    self.driver.get(cb_link)
                    r = self.driver.page_source
                    soup = bs(r, 'html5lib')
                    #print('cb_link')
                    try:
                        #Number of Funding Rounds
                        numFundRounds = soup.find_all('span', attrs={'class': 'bigValueItemLabelOrData flex-none layout-column layout-align-center-start'})[0].getText()
                        numFundRounds = numFundRounds.strip()
                        #print('Number of Funding Rounds: ' + numFundRounds)
                        self.numRounds.append(numFundRounds)
                        
                        #Total Funding Amount
                        totFundAmt = soup.find_all('span', attrs={'class': 'bigValueItemLabelOrData flex-none layout-column layout-align-center-start'})[1].getText()
                        totFundAmt = totFundAmt.strip()
                        #print('Total Funding Amount: ' + totFundAmt)
                        self.totAmt.append(totFundAmt)
                        
                        #Funding ID Parameter: Indicates whether data was pulled from Crunchbase, LN, or NA
                        #1 indicates Crunchbase
                        self.fundID.append(1)
                        #c = c + 1
                    except:
                        try:
                            self.driver.get(url)
                            r = self.driver.page_source
                            soup = bs(r, 'html5lib')
                            #Number of Funding Rounds
                            numFundRounds = soup.find('div', attrs={'class': 'org-funding__links t-14 t-black--light t-normal'}).getText()
                            numFundRounds = numFundRounds.strip()
                            #print('Number of Funding Rounds: ' + numFundRounds)
                            self.numRounds.append(numFundRounds)

                            #Total Funding Amount
                            totFundAmt = soup.find('span', attrs={'class': 't-24 t-light t-black--light'}).getText()
                            totFundAmt = totFundAmt.strip()
                            #print('Total Funding Amount: ' + totFundAmt)
                            self.totAmt.append(totFundAmt)
                            
                            #Funding ID Parameter: Indicates whether data was pulled from Crunchbase, LN, or NA
                            #2 indicates LinkedIn
                            self.fundID.append(2)
                            #c = c + 1
                        except:
                            self.numRounds.append('Not Available')
                            self.totAmt.append('Not Available')
                            
                            #Funding ID Parameter: Indicates whether data was pulled from Crunchbase, LN, or NA
                            #0 indicates Not Available
                            self.fundID.append(0)
                            #c = c + 1
                except:
                    try:
                        self.driver.get(url)
                        r = self.driver.page_source
                        soup = bs(r, 'html5lib')
                        #Number of Funding Rounds
                        numFundRounds = soup.find('div', attrs={'class': 'org-funding__links t-14 t-black--light t-normal'}).getText()
                        numFundRounds = numFundRounds.strip()
                        #print('Number of Funding Rounds: ' + numFundRounds)
                        self.numRounds.append(numFundRounds)
                        
                        #Total Funding Amount
                        totFundAmt = soup.find('span', attrs={'class': 't-24 t-light t-black--light'}).getText()
                        totFundAmt = totFundAmt.strip()
                        #print('Total Funding Amount: ' + totFundAmt)
                        self.totAmt.append(totFundAmt)
                        
                        #Funding ID Parameter: Indicates whether data was pulled from Crunchbase, LN, or NA
                        #2 indicates LinkedIn
                        self.fundID.append(2)
                        #c = c + 1
                    except:
                        self.numRounds.append('Not Available')
                        self.totAmt.append('Not Available')
                        
                        #Funding ID Parameter: Indicates whether data was pulled from Crunchbase, LN, or NA
                        #0 indicates Not Available
                        self.fundID.append(0)
                        #c = c + 1

In [3]:
def main():
    Scrape(0)
    
if __name__ == "__main__":
    main()

Enter Company URL(s): https://www.linkedin.com/company/hive9/about/ https://www.linkedin.com/company/parkmycloud/about/ 
Select 1 to export current data to CSV | 2 to add onto an existing CSV | Anything else to exit
1
          Name                                                URL      City  \
0        Hive9      https://www.linkedin.com/company/hive9/about/    Austin   
1  ParkMyCloud  https://www.linkedin.com/company/parkmycloud/a...  Sterling   

      State  Geo ID Founding Year Number of Employees  \
0     Texas       0          2015     11-50 employees   
1  Virginia       0          2015      2-10 employees   

                        Number of LinkedIn Employees  \
0  17 on LinkedIn\n            \n  \n\n\n\n    In...   
1  13 on LinkedIn\n            \n  \n\n\n\n    In...   

                                            Overview           Industry  \
0  Hive9 is the marketing performance management ...  Computer Software   
1  ParkMyCloud, a Turbonomic company, provides an... 